# Customer Lifetime Value

In [1]:
#Client id
id = '31ad1d1b63eb9962463f764d4e6e0c9d'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
orders = pd.read_csv('../dataset/olist_orders_dataset.csv', delimiter = ',')
customers = pd.read_csv('../dataset/olist_customers_dataset.csv', delimiter = ',')
reviews = pd.read_csv('../dataset/olist_order_reviews_dataset.csv', delimiter = ',')
payment = pd.read_csv('../dataset/olist_order_payments_dataset.csv', delimiter = ',')
order_items = pd.read_csv('../dataset/olist_order_items_dataset.csv', delimiter = ',')
products = pd.read_csv('../dataset/olist_products_dataset.csv', delimiter = ',')

named_items = pd.merge(order_items, products, on='product_id')
named_items = named_items[['order_id', 'price', 'freight_value', 'product_category_name']]

final = pd.merge(orders, customers, on='customer_id')
final = pd.merge(final, reviews, on='order_id')
#final = pd.merge(final, payment, on='order_id')
final = pd.merge(final, named_items, on='order_id')

client = final.loc[(final['customer_id'] == id)]

client = client.drop_duplicates()

client = client[['order_id','order_status', 'customer_city', 'customer_state', 
                 'review_score','price', 'freight_value', 'product_category_name', 
                 'order_delivered_customer_date', 'order_estimated_delivery_date',
                 'order_purchase_timestamp']]

client['order_delivered_customer_date'] = pd.to_datetime(client['order_delivered_customer_date'])
client['order_estimated_delivery_date'] = pd.to_datetime(client['order_estimated_delivery_date'])
client['order_purchase_timestamp'] = pd.to_datetime(client['order_purchase_timestamp'])


client['is_delayed'] = client['order_delivered_customer_date'] > client['order_estimated_delivery_date']
client['time_delay'] = client['order_delivered_customer_date'] - client['order_estimated_delivery_date']
client['time_to_wait'] = client['order_estimated_delivery_date'] - client['order_purchase_timestamp']
client['time_waited'] = client['order_delivered_customer_date'] - client['order_purchase_timestamp']

favorite_category = client.groupby(["product_category_name"]).size().reset_index(name="amount")

client['total'] = client['price'] + client['freight_value'] 

m = client.mean()

print('Customer average review score:', ("%.2f" % m[0]))
print('Average item price:', ("%.2f" % m[1]),'R$')
print('Average freight price:', ("%.2f" % m[2]),'R$')
print('Average delay:', ("%.2f" % (m[3]*100)), '%')

average_order = client.groupby('order_id', as_index=False)['price'].sum()

print('Average order price:', ("%.2f" % average_order.mean()[0]),'R$')

# region analisis
city_average_revenue = final.groupby('customer_city', as_index=False)['price'].mean()
state_average_revenue = final.groupby('customer_state', as_index=False)['price'].mean()

city = city_average_revenue.loc[(city_average_revenue['customer_city'] == client.iloc[0]['customer_city'])]
state = state_average_revenue.loc[(state_average_revenue['customer_state'] == client.iloc[0]['customer_state'])]


print('Client\'s city average revenue by client:', ("%.2f" % (city.iloc[0]['price'])),'R$')
print('Client\'s state average revenue by client:', ("%.2f" % (state.iloc[0]['price'])),'R$')

Customer average review score: 5.00
Average item price: 149.99 R$
Average freight price: 19.77 R$
Average delay: 0.00 %
Average order price: 149.99 R$
Client's city average revenue by client: 108.70 R$
Client's state average revenue by client: 109.58 R$


# Client's most bought categories

In [3]:
favorite_category.head()

,product_category_name,amount
0,moveis_escritorio,1


In [4]:
city.head()

,customer_city,price
3749,sorocaba,108.704986


In [5]:
state.head()

,customer_state,price
25,SP,109.579219


In [6]:
final.head(10)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,price,freight_value,product_category_name
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,29.99,8.72,utilidades_domesticas
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,BA,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,118.70,22.76,perfumaria
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,GO,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,159.90,19.22,automotivo
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,...,RN,359d03e676b3c069f62cadba8dd3f6e8,5,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58,45.00,27.20,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,...,SP,e50934924e227544ba8246aeb3770dd4,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51,19.90,8.72,papelaria
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01 00:00:00,80bb27c7c16e8f973207a5086ab329e2,86320,...,PR,89b738e70a1ce346db29a20fb2910161,4,NaN,NaN,2017-07-27 00:00:00,2017-07-27 22:48:30,147.90,27.36,automotivo
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00,36edbb3fb164b1f16485364b6fb04c73,98900,...,RS,e07549ef5311abcc92ba1784b093fb56,2,NaN,fiquei triste por n ter me atendido.,2017-05-13 00:00:00,2017-05-13 20:25:42,49.90,16.05,NaN
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07 00:00:00,932afa1e708222e5821dac9cd5db4cae,26525,...,RJ,07d67dd06ed5f88bef11ef6b464e79ae,5,NaN,NaN,2017-05-27 00:00:00,2017-05-28 02:59:57,59.99,15.17,automotivo
8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-01-25 02:50:47,2017-01-26 14:16:31,2017-02-02 14:08:10,2017-03-06 00:00:00,39382392765b6dc74812866ee5ee92a7,99655,...,RS,fc4af8aea8ec3f1a3cd181d3d0cadbd5,1,NaN,NaN,2017-02-03 00:00:00,2017-02-05 01:58:35,19.90,16.05,moveis_decoracao
9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29 11:55:02,2017-07-29 12:05:32,2017-08-10 19:45:24,2017-08-16 17:14:30,2017-08-23 00:00:00,299905e3934e9e181bfb2e164dd4b4f8,18075,...,SP,abc5655186d40772bd6e410420e6a3ed,5,NaN,NaN,2017-08-17 00:00:00,2017-08-18 01:47:32,149.99,19.77,moveis_escritorio
